## Gas

In [10]:
import gurobipy as gp

model = gp.Model('gas_fuel')

# Number of raw gas types and fuel blend types
n_raw_gas = 4
n_fuel_blend = 3

# Decision variables for the amount of each raw gas type to buy for each fuel blend type
gas_for_fuel = {}
for i in range(1, n_raw_gas + 1):
    for j in range(1, n_fuel_blend + 1):
        gas_for_fuel[(i, j)] = model.addVar(vtype=gp.GRB.CONTINUOUS, 
                                            name=f'gas{i}_for_fuel{j}')

# Decision variables for the amount of each raw gas type to buy for resale
gas_for_resale = {}
for i in range(1, n_raw_gas + 1):
    gas_for_resale[i] = model.addVar(vtype=gp.GRB.CONTINUOUS, 
                                     name=f'gas{i}_for_resale')
# Number of barrel of raw gas constraints
num_barrel_g1 = model.addConstr(gas_for_fuel[(1,1)] + 
                                gas_for_fuel[(1,2)] + 
                                gas_for_fuel[(1,3)] +
                                gas_for_resale[1] <= 4000, 'num_barrel_gas1')

num_barrel_g2 = model.addConstr(gas_for_fuel[(2,1)] + 
                                gas_for_fuel[(2,2)] + 
                                gas_for_fuel[(2,3)] +
                                gas_for_resale[2]<= 5050, 'num_barrel_gas1')

num_barrel_g3 = model.addConstr(gas_for_fuel[(3,1)] + 
                                gas_for_fuel[(3,2)] + 
                                gas_for_fuel[(3,3)] +
                                gas_for_resale[3]<= 7100, 'num_barrel_gas1')

num_barrel_g4 = model.addConstr(gas_for_fuel[(4,1)] + 
                                gas_for_fuel[(4,2)] + 
                                gas_for_fuel[(4,3)] +
                                gas_for_resale[4]<= 4300, 'num_barrel_gas1')

# Number of barrel of fuel blend constraints
# Constraints for all three fuel blends
upper = [10000, 999999,999999]
lower = [0, 0, 15000]
for j in range(1, n_fuel_blend + 1):
    # Upper bound constraint
    model.addConstr(
        sum(gas_for_fuel[(i,j)] for i in range(1, n_raw_gas + 1))
        <= upper[j-1], f'num_barrel_gas_upper_blend{j}')
    
    # Lower bound constraint
    model.addConstr(
        sum(gas_for_fuel[(i,j)] for i in range(1, n_raw_gas + 1))
        >= lower[j-1], f'num_barrel_gas_lower_blend{j}')


# Octane ratings for each raw gas type
octane_ratings = [68, 86, 91, 99]

# Minimum octane ratings for each fuel blend type
min_octane_fuel_blend = [95, 90, 85]

for j in range(1, n_fuel_blend + 1):
    # Calculate the total weighted octane for the fuel blend
    total_weighted_octane = sum(octane_ratings[i-1] * gas_for_fuel[(i,j)] for i in range(1, n_raw_gas + 1))
    
    # Calculate the total amount of the fuel blend
    total_fuel_blend = sum(gas_for_fuel[(i,j)] for i in range(1, n_raw_gas + 1))
    
    # Add constraint to ensure the average octane rating meets the minimum requirement
    model.addConstr(total_weighted_octane >= total_fuel_blend * min_octane_fuel_blend[j-1], f'octane_blend{j}')

# Maximize profit
# Cost and resale price for each raw gas type
cost_resale = [(31.02, 36.85), (33.15, 36.85), (36.35, 38.95), (38.7, 38.95)]

# Sale prices for each fuel blend type
sale_prices_fuel_blend = [45.15, 42.95, 40.99]

profit_resale = sum((cost_resale[i-1][1] - cost_resale[i-1][0]) * gas_for_resale[i] for i in range(1, n_raw_gas + 1))

# Correcting the profit_fuel_blend calculation
profit_fuel_blend = sum(
    sale_prices_fuel_blend[j-1] * sum(gas_for_fuel[(i,j)] for i in range(1, n_raw_gas + 1)) 
    - sum(cost_resale[i-1][0] * gas_for_fuel[(i,j)] for i in range(1, n_raw_gas + 1)) 
    for j in range(1, n_fuel_blend + 1))

# Set the objective to be the sum of the profits from resale and fuel blends
model.setObjective(profit_resale + profit_fuel_blend, gp.GRB.MAXIMIZE)

model.optimize()

# Check the solution
for v in model.getVars():
    print('%s : %g' % (v.varName, v.x))

print('Total Profit : %g' % model.objVal)


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 9 4900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 16 columns and 52 nonzeros
Model fingerprint: 0xb816cd3c
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e-01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+03, 1e+06]
Presolve removed 6 rows and 5 columns
Presolve time: 0.01s
Presolved: 7 rows, 11 columns, 31 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4493500e+05   1.197398e+04   0.000000e+00      0s
       5    1.4043148e+05   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.404314815e+05
gas1_for_fuel1 : 633.214
gas1_for_fuel2 : 0
gas1_for_fuel3 : 2824.19
gas2_for_fuel1 : 0
gas2_for_fuel2 : 0
gas2_for_fuel3 : 5050
gas3_for_fuel

## Nurse

In [19]:
model = gp.Model('nurse')

# Create the decision variables
# Possible shifts
# 12 AM - 8 AM
# 4 AM - 12 PM
# 8 AM - 4 PM
# 12 PM - 8 PM
# 4 PM - 12 AM
# 8 PM - 4 AM

s1 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='shift1')
s2 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='shift2')
s3 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='shift3')
s4 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='shift4')
s5 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='shift5')
s6 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='shift6')

# Create the constraints
slot1 = model.addConstr(s1 + s6 >=4, 'slot1')
slot2 = model.addConstr(s1 + s2 >=8, 'slot2')
slot3 = model.addConstr(s2 + s3 >=10, 'slot3')
slot4 = model.addConstr(s3 + s4 >=7, 'slot4')
slot5 = model.addConstr(s4 + s5 >=12, 'slot5')
slot6 = model.addConstr(s5 + s6 >=4, 'slot6')

# Create the objective function
model.setObjective(s1+s2+s3+s4+s5+s6, gp.GRB.MINIMIZE)

# Extra material. You don't need them, but it's nice to know what they do!
model.write(model.ModelName + '.lp')
model.setParam('OutputFlag', 0)

# Solve the model
model.optimize()

# Check the solution
for v in model.getVars():
    print('%s : %g' % (v.varName, v.x))

print('Total Nurses : %g' % model.objVal)



shift1 : 4
shift2 : 10
shift3 : 0
shift4 : 8
shift5 : 4
shift6 : 0
Total Nurses : 26


12 AM - 8 AM: 4 nurses

4 AM - 12 PM: 10 nurses

8 AM - 4 PM: 0 nurses

12 PM - 8 PM: 8 nurses

4 PM - 12 AM: 4 nurses

8 PM - 4 AM: 0 nurses

The sum of the number of nurses of the overlapping shifts should be at least the minimum number of nurses for the overlapping time slot

## Cutting Stock

In [20]:
model = gp.Model('stock')


# Decision Variables: All possible ways to cut 20ft wasting the least

x1 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='4x 5ft')
x2 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='2x 5ft 1x 7ft')
x3 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='2x 5ft 1x 9ft')
x4 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='2x 7ft 1x5 ft')
x5 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='1x 7ft 1x 9ft')
x6 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='2x 9ft')

d1 = model.addConstr(4*x1 + 2*x2 + 2*x3 + x4 >= 150, 'd1')
d2 = model.addConstr(x2 + 2*x4 + x5 >= 200, 'd2')
d2 = model.addConstr(x3 + x5 + 2*x6>= 300, 'd3')

# Create the objective function
model.setObjective(x1+x2+x3+x4+x5+x6, gp.GRB.MINIMIZE)

# Extra material. You don't need them, but it's nice to know what they do!
model.write(model.ModelName + '.lp')
model.setParam('OutputFlag', 0)

# Solve the model
model.optimize()

# Check the solution
for v in model.getVars():
    print('%s : %g' % (v.varName, v.x))

print('Total Rolls : %g' % model.objVal)

4x 5ft : 12.5
2x 5ft 1x 7ft : 0
2x 5ft 1x 9ft : 0
2x 7ft 1x5 ft : 100
1x 7ft 1x 9ft : 0
2x 9ft : 150
Total Rolls : 262.5
